In [1]:
import pandas as pd

In [ ]:
CALL gds.louvain.write('locationComplaints',{
    includeIntermediateCommunities: true,
    writeProperty: 'louvain'
})
YIELD ranLevels, communityCount, modularity,modularities

In [ ]:
CALL gds.louvain.stream('locationComplaints')
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).location AS name, communityId, intermediateCommunityIds
ORDER BY communityId ASC

In [ ]:
CALL gds.pageRank.write('locationPageRank',{
    writeProperty: 'pagerank'
})
YIELD nodePropertiesWritten, ranIterations

In [ ]:
match(l:Location) 
with l.louvain[-1] as community, max(l.pagerank) as pagerank
match(l:Location)
where l.pagerank = pagerank and l.louvain[-1] = community
return l.location_id as location_id, apoc.convert.toFloat(l.latitude) as latitude, apoc.convert.toFloat(l.longitude) as longitude, community

In [5]:
community = pd.read_csv('./data/clusters/community_points.csv')
community

,location_id,latitude,longitude,community
0,17354,40.667386,-73.854055,26010
1,38710,40.739935,-73.853203,13782
2,57811,40.857886,-73.904778,28417
3,50932,40.815101,-73.861018,35189
4,58276,40.862096,-73.824818,20875
...,...,...,...,...
157,2554,40.586727,-74.128641,10551
158,5906,40.609865,-74.188710,10612
159,42918,40.755767,-73.956637,10818
160,4930,40.603518,-74.105298,12117


In [6]:
community_size = pd.read_csv('./data/clusters/community_size.csv')
community_size

,community,community_size
0,26010,2556
1,13782,2223
2,28417,1735
3,35189,1685
4,20875,1970
...,...,...
129,10551,1
130,10612,1
131,10818,1
132,12117,1


In [11]:
community_points = community.merge(community_size, how='outer', on='community')

In [12]:
community_points.to_csv('./data/clusters/community_points.csv')